In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import numpy as np
import pandas as pd 

data_path = '/content/drive/MyDrive/ solar power/'

train = pd.read_csv(data_path + 'train/train.csv')

In [13]:
#hour angle 추가
h_list=[]
len(train['Hour'])
for i in range(len(train['Hour'])):
  h=train.loc[i,'Hour']
  min=train.loc[i,'Minute']
  if min!=30:
    hour_angle=15*(h-12)-7.5
    h_list.append(hour_angle)
  else:
    hour_angle=15*(h-12)
    h_list.append(hour_angle)

train['h']=h_list

In [18]:
train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time
0,0,0,0,0,0,1.5,69.08,-12,0.0,-187.5,0
1,0,0,30,0,0,1.5,69.06,-12,0.0,-180.0,30
2,0,1,0,0,0,1.6,71.78,-12,0.0,-172.5,60
3,0,1,30,0,0,1.6,71.75,-12,0.0,-165.0,90
4,0,2,0,0,0,1.6,75.20,-12,0.0,-157.5,120


In [15]:
# Time 변수 생성 
train['Time'] = train['Hour'] * 60 + train['Minute']
train.to_csv('train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52560 entries, 0 to 52559
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Day     52560 non-null  int64  
 1   Hour    52560 non-null  int64  
 2   Minute  52560 non-null  int64  
 3   DHI     52560 non-null  int64  
 4   DNI     52560 non-null  int64  
 5   WS      52560 non-null  float64
 6   RH      52560 non-null  float64
 7   T       52560 non-null  int64  
 8   TARGET  52560 non-null  float64
 9   h       52560 non-null  float64
 10  Time    52560 non-null  int64  
dtypes: float64(4), int64(7)
memory usage: 4.4 MB


In [17]:
train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time
0,0,0,0,0,0,1.5,69.08,-12,0.0,-187.5,0
1,0,0,30,0,0,1.5,69.06,-12,0.0,-180.0,30
2,0,1,0,0,0,1.6,71.78,-12,0.0,-172.5,60
3,0,1,30,0,0,1.6,71.75,-12,0.0,-165.0,90
4,0,2,0,0,0,1.6,75.20,-12,0.0,-157.5,120


In [19]:
# 날짜별 데이터프레임 쪼개기
train_day = []
list_day = [] 
DAY = 0

for n in range(0,train.shape[0]):

    if train.loc[n, 'Day'] == DAY:
        list_day.append(train.loc[n, ['Time','TARGET']])
        
    else:
        day = pd.DataFrame(list_day, columns = ['Time', 'TARGET'], index = range(0,48))
        train_day.append(day)

        list_day = []
        list_day.append(train.loc[n, ['Time','TARGET']])
    DAY = train.loc[n, 'Day']



In [26]:
print(train_day)

[      Time     TARGET
0      0.0   0.000000
1     30.0   0.000000
2     60.0   0.000000
3     90.0   0.000000
4    120.0   0.000000
5    150.0   0.000000
6    180.0   0.000000
7    210.0   0.000000
8    240.0   0.000000
9    270.0   0.000000
10   300.0   0.000000
11   330.0   0.000000
12   360.0   0.000000
13   390.0   0.000000
14   420.0   0.000000
15   450.0   0.000000
16   480.0   7.039287
17   510.0   5.912871
18   540.0  22.337268
19   570.0  29.469529
20   600.0  25.339762
21   630.0  25.152060
22   660.0  28.718397
23   690.0  33.129393
24   720.0  19.427151
25   750.0  25.715166
26   780.0  24.589225
27   810.0  21.304405
28   840.0  11.731500
29   870.0  14.734764
30   900.0   5.818888
31   930.0   7.602096
32   960.0   4.035725
33   990.0   0.938541
34  1020.0   0.000000
35  1050.0   0.000000
36  1080.0   0.000000
37  1110.0   0.000000
38  1140.0   0.000000
39  1170.0   0.000000
40  1200.0   0.000000
41  1230.0   0.000000
42  1260.0   0.000000
43  1290.0   0.000000
44  1320.

In [27]:
# 일출시간, 일몰시간 구하기
new = pd.DataFrame(columns = ['Day', '일출시간', '일몰시간', '일조시간'])

for i in range(0,len(train_day)):
    for d1 in range(0,48):
        if train_day[i].loc[d1,'TARGET'] != 0:
            sunrise = train_day[i].loc[d1,'Time']
            break

    for d2 in range(47,-1,-1):
        if train_day[i].loc[d2,'TARGET'] != 0:
            sunset = train_day[i].loc[d2 + 1,'Time']
            break
        
    new.loc[i] = [i, sunrise, sunset, sunset - sunrise]

new.to_csv('new.csv')

In [28]:
new_train = pd.read_csv('new.csv')

In [30]:
new_train.head()

,Unnamed: 0,Day,일출시간,일몰시간,일조시간
0,0,0.0,480.0,1020.0,540.0
1,1,1.0,480.0,1020.0,540.0
2,2,2.0,480.0,1020.0,540.0
3,3,3.0,480.0,1020.0,540.0
4,4,4.0,480.0,1020.0,540.0


In [31]:
# 하지, 동지 구하기

summer = new.loc[new['일조시간'].idxmax(), 'Day']
winter = new.loc[new['일조시간'].idxmin(), 'Day']

print(summer, winter)

# 138 + 195 = 233 -> 1월 1일

138.0 356.0


In [32]:
N=[]
#1월 1일 기준 N+171 = 하지 기준 N
for i in range(len(train['Day'])):
  day=train.loc[i,'Day']
  if day==summer:
    N.append(+171)
  else:
    N.append(day-summer+171)


#N Feature 생성
#N이 음수, 364 초과를 변환(1월 1일 기준 N=0)
train['N']=N
day_max=train['N'].max()
day_min=train['N'].min()

while day_max>364 or day_max<0:
  for i in range(len(train['N'])):
    day=train.loc[i,'N']
    if day<0:
      train.loc[i,'N']=day+365
    elif day>364:
      train.loc[i,'N']=day-365
    else:
      pass
  day_max=train['N'].max()
  day_min=train['N'].min()

#현재 N은 한국 기준이므로, UT로 변환
train['N']-=0.375


In [33]:
#Declination of the Sun

def radian_angle(x):
  return x*np.pi/180

def dec_sun(N):
  return -1*np.arcsin(0.39779*np.cos(radian_angle(0.98565)*(N+10)+radian_angle(1.914)*np.sin(radian_angle(0.98565)*(N-2))))

train['Declination']=dec_sun(train['N'])

In [34]:
train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination
0,0,0,0,0,0,1.5,69.08,-12,0.0,-187.5,0,32.625,-0.295314
1,0,0,30,0,0,1.5,69.06,-12,0.0,-180.0,30,32.625,-0.295314
2,0,1,0,0,0,1.6,71.78,-12,0.0,-172.5,60,32.625,-0.295314
3,0,1,30,0,0,1.6,71.75,-12,0.0,-165.0,90,32.625,-0.295314
4,0,2,0,0,0,1.6,75.20,-12,0.0,-157.5,120,32.625,-0.295314


In [36]:
#Zenith Angle

#관측자 위도 설정
alt=37.54

def zenith_angle(altitude,declination,hour_angle):
  return np.sin(radian_angle(altitude))*np.sin(declination)+np.cos(radian_angle(altitude))*np.cos(declination)*np.cos(radian_angle(hour_angle))

train['Zenith_Angle']=zenith_angle(alt,train['Declination'],train['h'])

In [37]:
train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination,Zenith_Angle
0,0,0,0,0,0,1.5,69.08,-12,0.0,-187.5,0,32.625,-0.295314,-0.929448
1,0,0,30,0,0,1.5,69.06,-12,0.0,-180.0,30,32.625,-0.295314,-0.935938
2,0,1,0,0,0,1.6,71.78,-12,0.0,-172.5,60,32.625,-0.295314,-0.929448
3,0,1,30,0,0,1.6,71.75,-12,0.0,-165.0,90,32.625,-0.295314,-0.910089
4,0,2,0,0,0,1.6,75.20,-12,0.0,-157.5,120,32.625,-0.295314,-0.878193


In [ ]:
#GHI

train['GHI']=train['DHI']+train['DNI']*train['Zenith_Angle']

        Day  Hour  Minute    DHI    DNI   WS      RH     T     TARGET      h  \
6624  138.0   0.0     0.0    0.0    0.0  3.5  100.00   6.0   0.000000 -187.5   
6625  138.0   0.0    30.0    0.0    0.0  3.3  100.00   5.0   0.000000 -180.0   
6626  138.0   1.0     0.0    0.0    0.0  3.1   99.26   5.0   0.000000 -172.5   
6627  138.0   1.0    30.0    0.0    0.0  3.0  100.00   5.0   0.000000 -165.0   
6628  138.0   2.0     0.0    0.0    0.0  3.0  100.00   5.0   0.000000 -157.5   
6629  138.0   2.0    30.0    0.0    0.0  2.9  100.00   5.0   0.000000 -150.0   
6630  138.0   3.0     0.0    0.0    0.0  2.9  100.00   5.0   0.000000 -142.5   
6631  138.0   3.0    30.0    0.0    0.0  2.4  100.00   4.0   0.000000 -135.0   
6632  138.0   4.0     0.0    0.0    0.0  1.9  100.00   4.0   0.000000 -127.5   
6633  138.0   4.0    30.0    0.0    0.0  1.9  100.00   3.0   0.000000 -120.0   
6634  138.0   5.0     0.0    4.0    0.0  1.9  100.00   3.0   0.375379 -112.5   
6635  138.0   5.0    30.0   20.0    0.0 